In [1]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils.multiclass import unique_labels
import itertools
from scipy import misc
import math
import os
import glob
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import pylab as plt
import numpy as np
from scipy.signal import argrelextrema
from IPython.display import Image
import matplotlib.patches as patches
import matplotlib as matplotlib
import xlsxwriter
from xlrd import open_workbook
from xlutils.copy import copy
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from datetime import datetime
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd

In [2]:
filename='/Users/JoyThompson/Desktop/Forecasting CK Budget/Budget Analysis Flat Rate Increase.xlsx'

bills=pd.read_excel(filename, sheetname='forecast2020')
rates= pd.read_excel(filename, sheetname='Flatrates')
rates=rates.set_index('Client')
print(bills)

               Client Name    2020-01-01   2020-02-01   2020-03-01  \
0               3B Medical  30986.850000  31403.50000  31820.15000   
1                 Ambreesh   3000.000000   3000.00000   3000.00000   
2                  BHF-USA   8215.810000   4621.39000   3388.93000   
3              Brain Armor   1000.000000   3000.00000   3000.00000   
4               Bright Fox   2000.000000   2000.00000   2000.00000   
5                  Coyuchi   3000.000000   3000.00000   3000.00000   
6             Dammit Dolls   2000.000000   2000.00000   2000.00000   
7        David's Wholesale  20087.415000  20598.26250  22307.75250   
8                  DaVinci   3000.000000   3000.00000   3000.00000   
9                   Dedeco   3000.000000   3000.00000   3000.00000   
10      Fast Growing Trees   3000.000000   3000.00000   4250.00000   
11             Forever New   4000.000000   4000.00000   4000.00000   
12             geckobrands   2000.000000   2000.00000   5000.00000   
13                  

In [3]:
print(bills.loc[2, 'Contract End Date'])
for ind in bills.index:
    enddate=bills.loc[ind, 'Contract End Date']
    newdate=pd.to_datetime(enddate)
    bills.loc[ind, 'Contract End Date']=newdate
print(type(bills.loc[2, 'Contract End Date']))


2020-07-31 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [4]:
numbill=len(bills.columns)
print(numbill)
clientsa=bills.set_index('Client Name')
clientsall=clientsa.index
datecol=bills.columns
datecol=datecol[1:-1]
datecol2=[]
for date in datecol:
    print(date)
    newdate=pd.to_datetime(date)
    datecol2.append(newdate)
print(clientsall)
print(datecol2)

14
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
Index(['3B Medical', 'Ambreesh', 'BHF-USA', 'Brain Armor', 'Bright Fox',
       'Coyuchi', 'Dammit Dolls', 'David's Wholesale', 'DaVinci', 'Dedeco',
       'Fast Growing Trees', 'Forever New', 'geckobrands', 'Geox', 'Ginsu',
       'Indosole', 'Innova', 'Jax Wax', 'Juggernaut', 'Knit Rite (Therafirm)',
       'Little Giraffe', 'Marula Oil', 'MasqueBar', 'Mavi', 'Molex',
       'Montana West', 'Nano Film', 'Not Rated', 'Organyc',
       'Outdoor Hub (Five Ten)', 'P.E. Nation', 'Pet 'n Shape', 'Peter Grimm',
       'Rago', 'Sassy Lashes', 'Snif Snax', 'Soludos', 'Spin Life', 'Westmoor',
       'Wine Enthusiast', 'Workman's Friend', 'Worldbook', 'New Account 1',
       'New Account 2', 'New Account 3', 'New Account 4', 'New Account 5',
       'New Account 6', 'New Account 7', 'New Account 8', 'New Account 9',
       'New Account 10', 'New Account 11', 'New

In [5]:
def flatrateincrease(origbill, increase):
    billsnew=pd.DataFrame(index=clientsall, columns=datecol2)
    for ind in bills.index:
        client=bills.loc[ind, 'Client Name']
        contend=bills.loc[ind, 'Contract End Date']
        for indbill in range(1, numbill-1):
            val=bills.iloc[ind, indbill]
            dateind=indbill-1
            date=datecol2[dateind]
            flatrate=rates.loc[client, 'Flatrate']

            weird=rates.loc[client, 'IsWeird']
            
      
            if contend<=date:
                if weird == 'No':
                    if val==flatrate:
                        newval=flatrate*(1+increase)
                        billsnew.loc[client, date]=newval
                    else:
                         billsnew.loc[client, date]=val
                        

                if weird == 'Yes':
                    newval=increase*flatrate+val
                    billsnew.loc[client, date]=newval
            
            else:
             
                billsnew.loc[client, date]=val

    return billsnew

In [6]:
bills5per=flatrateincrease(bills, .05)
print(bills5per)

                       2020-01-01 2020-02-01 2020-03-01 2020-04-01 2020-05-01  \
Client Name                                                                     
3B Medical                30986.9    31403.5    31820.2    32236.8    32653.5   
Ambreesh                     3000       3000       3000       3000       3000   
BHF-USA                   8215.81    4621.39    3388.93     2464.7    2567.44   
Brain Armor                  1000       3000       3000       3000       3150   
Bright Fox                   2100       2100       2100       2100       2100   
Coyuchi                      3000       3000       3000       3000       3150   
Dammit Dolls                 2000       2000       2000       2000       2000   
David's Wholesale         20087.4    20598.3    22307.8    22493.4    22679.6   
DaVinci                      3000       3000       3000       3150       3150   
Dedeco                       3150       3150       3150       3150       3150   
Fast Growing Trees          

In [7]:
bills10per=flatrateincrease(bills, .10)
bills15per=flatrateincrease(bills, .15)
print(bills10per)

                       2020-01-01 2020-02-01 2020-03-01 2020-04-01 2020-05-01  \
Client Name                                                                     
3B Medical                30986.9    31403.5    31820.2    32236.8    32653.5   
Ambreesh                     3000       3000       3000       3000       3000   
BHF-USA                   8215.81    4621.39    3388.93     2464.7    2567.44   
Brain Armor                  1000       3000       3000       3000       3300   
Bright Fox                   2200       2200       2200       2200       2200   
Coyuchi                      3000       3000       3000       3000       3300   
Dammit Dolls                 2000       2000       2000       2000       2000   
David's Wholesale         20087.4    20598.3    22307.8    22493.4    22679.6   
DaVinci                      3000       3000       3000       3300       3300   
Dedeco                       3300       3300       3300       3300       3300   
Fast Growing Trees          

In [8]:
filenamefinal = '/Users/JoyThompson/Desktop/Forecasting CK Budget/Flatrateincreasecurrentbills.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
bills5per.to_excel(writer, sheet_name='Five Percent Increase')
bills10per.to_excel(writer, sheet_name='Ten Percent Increase')
bills15per.to_excel(writer, sheet_name='Fifteen Percent Increase')
writer.save()